In [1]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers==4.28.0 datasets



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import random
from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from IPython.display import display, HTML


In [5]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [6]:

# Load and preprocess the dataset
def preprocess_function(examples):
    out = [tokenizer(example,
                     padding="max_length",
                     truncation=True,
                     max_length=512).input_ids for example in examples] 
    return out

go_emotions = load_dataset('go_emotions')

train_dataset = Dataset.from_dict({
    'input_ids': preprocess_function(go_emotions['train']['text'][:100]),
    'labels': preprocess_function(go_emotions['train']['text'][:100])
})

valid_dataset = Dataset.from_dict({
    'input_ids': preprocess_function(go_emotions['validation']['text'][:50]),
    'labels': preprocess_function(go_emotions['validation']['text'][:50])
})


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
batch_size = 8
args = TrainingArguments(f"{model_name}-finetuned-repeat",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
)

In [8]:


# Train the model
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.103558


TrainOutput(global_step=13, training_loss=1.9944333296555738, metrics={'train_runtime': 606.7699, 'train_samples_per_second': 0.165, 'train_steps_per_second': 0.021, 'total_flos': 13534180147200.0, 'train_loss': 1.9944333296555738, 'epoch': 1.0})

In [9]:
trainer.save_model('./fine_tuned_model')
